In [1]:
import os
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize as tokenizer

nltk.download('punkt')
nltk.download('stopwords')

englishStopWords = stopwords.words('english')
stemmer = PorterStemmer()

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [2]:
mainDirectory = '/kaggle/input/quora-insincere-questions-classification/'

trainFile = mainDirectory + 'train.csv'
testFile = mainDirectory + 'test.csv'
submissionFile = mainDirectory + 'sample_submission.csv'

In [3]:
trainDf = pd.read_csv(trainFile, nrows = 100000)
trainDf

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
99995,13956982bc38015ca63f,What is it like to switch from Adobe Premiere ...,0
99996,1395736e4b14323fc16a,What will be the effect of long term capital g...,0
99997,139577538cc474df1aa0,"Which is greater, the need to eat or have sex?...",0
99998,13958561bd4190b383b9,Is there any way mechanical engineers can use ...,0


In [4]:
testDf = pd.read_csv(testFile)
submissionDf = pd.read_csv(submissionFile)

In [5]:
trainDf.target.value_counts(normalize = True)

0    0.93853
1    0.06147
Name: target, dtype: float64

In [6]:
submissionDf.prediction.value_counts()

0    375806
Name: prediction, dtype: int64

In [7]:
firstSentence = trainDf.question_text[0]
firstSentence

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [8]:
firstToken = tokenizer(firstSentence)
firstToken

['How',
 'did',
 'Quebec',
 'nationalists',
 'see',
 'their',
 'province',
 'as',
 'a',
 'nation',
 'in',
 'the',
 '1960s',
 '?']

In [9]:
", ".join(englishStopWords)

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [10]:
firstNonStopWords = [word for word in firstToken if word.lower() not in englishStopWords]
firstNonStopWords

['Quebec', 'nationalists', 'see', 'province', 'nation', '1960s', '?']

In [11]:
firstStemmingWords = [stemmer.stem(word) for word in firstNonStopWords]
firstStemmingWords

['quebec', 'nationalist', 'see', 'provinc', 'nation', '1960', '?']

In [12]:
[stemmer.stem(word) for word in tokenizer(firstSentence) if word.lower() not in englishStopWords]

['quebec', 'nationalist', 'see', 'provinc', 'nation', '1960', '?']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

def tokenize(self, text):
    return [stemmer.stem(word) for word in word_tokenize(text)]

In [14]:
vectorizer = CountVectorizer(lowercase = True, tokenizer = tokenizer, stop_words = englishStopWords, max_features = 1000)

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
%%time

vectorizer.fit(trainDf.question_text)

CPU times: user 13.2 s, sys: 36.4 ms, total: 13.2 s
Wall time: 13.3 s


CountVectorizer(max_features=1000,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function word_tokenize at 0x76bc35c07ef0>)

In [17]:
vectorizer.get_feature_names_out

<bound method CountVectorizer.get_feature_names_out of CountVectorizer(max_features=1000,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function word_tokenize at 0x76bc35c07ef0>)>

In [18]:
inputs = vectorizer.transform(trainDf.question_text)

In [19]:
inputs.shape

(100000, 1000)

In [20]:
test_inputs = vectorizer.transform(testDf.question_text)
test_inputs.shape

(375806, 1000)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_val, y_train, y_val = train_test_split(inputs.toarray(), trainDf.target, test_size = 0.3, random_state = 4)
print(len(X_train), len(X_val), len(y_train), len(y_val))

70000 30000 70000 30000


In [23]:
model = LogisticRegression(random_state=4, max_iter = 10000, solver = 'newton-cg')
model.fit(X_train, y_train)
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)
print(f"Traning Accuracy: {accuracy_score(train_preds, y_train)}")
print(f"Validation Accuracy: {accuracy_score(val_preds, y_val)}")
print(f"Traning F1 Score: {f1_score(train_preds, y_train)}")
print(f"Validation F1 Score: {f1_score(val_preds, y_val)}")

Traning Accuracy: 0.9474714285714285
Validation Accuracy: 0.9466
Traning F1 Score: 0.3941341242379305
Validation F1 Score: 0.3824209714726291


In [24]:
test_preds = model.predict(test_inputs)

In [25]:
submissionDf.prediction = test_preds
submissionDf.prediction.value_counts()

0    366278
1      9528
Name: prediction, dtype: int64

In [26]:
submissionDf.to_csv('submission.csv')